In [1]:
from datetime import datetime,timedelta
import time
import os
import requests
import cv2 as cv
import numpy as np

In [2]:
#creating a function that automatically saves the file to the necessary directory
def save_frame(frame, count, filler, frame_type = 'processed'):
    if frame_type == 'Original':
        savepath= r'C:\\Users\\Dani\\Downloads\\saving frames from jump rope video\\original'
        file_name = '{} {}.png'.format(filler, count)
        save_path = os.path.join(savepath,file_name)
        cv.imwrite(save_path, frame)
    else:
        savepath= r'C:\\Users\\Dani\\Downloads\\saving frames from jump rope video\\preprocessed'
        file_name = '{} {}.png'.format(filler, count)
        save_path = os.path.join(savepath,file_name)
        cv.imwrite(save_path, frame)
        
#creating a function that breaks down the videos frame by frame and uses the
#save_frame function to save them to the necessary directories
def dense_OF_farneback_preprocess_frames(video_path):
    capture = cv.VideoCapture(video_path)
    ret, old_frame = capture.read()
    flow = np.zeros((old_frame.shape[0], old_frame.shape[1], 2), dtype=np.float32)
    params4farne = {'pyr_scale':0.5,'levels':3, 'winsize':15,'iterations': 3, 'poly_n': 5,'poly_sigma':1.2,'flags':cv.OPTFLOW_USE_INITIAL_FLOW}
    counter = 1
    #in order to avoid duplicates files replacing each other
    #I gave each file a 'letter' that would be placed before their file names
    filler_digit = int(video_path[-5:-4]) - 1 
    alphabet = ['a','b','c','d','e','f', 'g','h','i', 'j', 'k', 'l', 'm']
    filler_digit = alphabet[filler_digit]
    hsv = np.zeros_like(old_frame)
    hsv[...,1] = 255
    first_gray = cv.cvtColor(old_frame,cv.COLOR_BGR2GRAY)
    while(1):
        ret1,current_frame = capture.read()
        if ret1:
            second_gray = cv.cvtColor(current_frame,cv.COLOR_BGR2GRAY)
            flow = cv.calcOpticalFlowFarneback(first_gray,second_gray,None,0.5, 3, 15, 3, 5, 1.2, 0)
            mag,angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
            hsv[...,0] = angle * 180 / np.pi/2
            hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
            RGB = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
            dims = (400,400)
            #First, save the RGB frames to the corresponding directory
            resized_frame_colored = cv.resize(RGB,dims,interpolation=cv.INTER_AREA)
            save_frame(frame = resized_frame_colored, count = counter,filler = filler_digit, frame_type = 'processed')
            #Next, save the 'original' (unprocessed) frames to the corresponding directory
            resized_frame_original = cv.resize(current_frame,dims,interpolation=cv.INTER_AREA)
            save_frame(frame = resized_frame_original, count = counter,filler = filler_digit, frame_type = 'Original')    
            first_gray = second_gray
            #making the program captures 50 frames per second
            if cv.waitKey(20) & 0xFF == ord('x'):
                break
            counter += 1
        else:
            #printing the total number of frames the video had
            print(counter)
            break

In [3]:
#I just put an example of a video I would process for demonstration
#purposes only
base_string = 'youtube_jumpropevid_11.mp4'
to_proc = r"C:\\Users\\Dani\\Downloads\\"
to_proc2 = to_proc + base_string
dense_OF_farneback_preprocess_frames(video_path=to_proc2)

931
